In [1]:
import torch
import sys
import os

# Append the parent directory (..) to sys.path
#sys.path.append('..')
#sys.path

In [2]:
sys.path.append(os.path.abspath('..'))

In [3]:
sys.path

['C:\\Users\\adhem\\Anaconda3\\Library\\bin',
 'c:\\Users\\adhem\\anaconda3\\envs\\Audio\\python311.zip',
 'c:\\Users\\adhem\\anaconda3\\envs\\Audio\\DLLs',
 'c:\\Users\\adhem\\anaconda3\\envs\\Audio\\Lib',
 'c:\\Users\\adhem\\anaconda3\\envs\\Audio',
 '',
 'c:\\Users\\adhem\\anaconda3\\envs\\Audio\\Lib\\site-packages',
 'c:\\Users\\adhem\\anaconda3\\envs\\Audio\\Lib\\site-packages\\win32',
 'c:\\Users\\adhem\\anaconda3\\envs\\Audio\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\adhem\\anaconda3\\envs\\Audio\\Lib\\site-packages\\Pythonwin',
 'c:\\Users\\adhem\\anaconda3\\envs\\Audio\\Lib\\site-packages\\setuptools\\_vendor',
 'c:\\Users\\adhem\\Desktop\\Music\\Custom_VST']

In [4]:
from models.autoencoders.encoder_1D import Autoencoder1d

In [5]:
from models.losses.factory import *

In [6]:
loss_cfg = {
    'L1TemporalLoss':{
        'weight':10,
        'balancer': False,
        'key_output': 'x_hat',
        'key_target': 'x',
        },
    'L2TemporalLoss':{
        'weight':100,
        'balancer': False,
        'key_output': 'x_hat',
        'key_target': 'x',
        },
    'AuralossLoss':{
        'weight':1,
        'balancer': False,
        'key_output': 'x_hat',
        'key_target': 'x',
        "fft_sizes": [32, 128, 512, 2048],  # [32, 128, 512, 2048, 8192, 32768]
        "hop_sizes": [16, 64, 256, 1024],  # [16, 64, 256, 1024, 4096, 16384]
        "win_lengths": [32, 128, 512, 2048],  # [32, 128, 512, 2048, 8192, 32768]
        "w_sc": 0.0,
        "w_phs": 0.0,
        "w_lin_mag": 1.0,
        "w_log_mag": 1.0,
    },
    #'KLDivergenceLoss':{
    #    'weight':0,
    #    'balancer': False,
    #    'key_mean': 'z_mean',
    #    'key_log_std': 'z_log_std',
    #    },
    'Discriminator':{
        'weight':5,
        'balancer': False,
        #'type':'Spectrogram_VGGStyle',
    },
    'FeatureLoss':{
        'weight':1,
        'balancer': False,
    },
}

In [7]:
test_loss = CraftLosses(**loss_cfg)

In [8]:
test_loss.nobalancer_config

{'generator_loss': 5,
 'feature_loss': 1,
 'l1_loss': 10,
 'l2_loss': 100,
 'aura_loss': 1}

In [9]:
test_loss.balancer_config

{}

In [10]:
model_config = {
    'in_channels': 1,
    'out_channels': 16,
    'channels': [16, 32, 64, 64],
    'factors': [4, 4, 4, 4],
    'res_blocks': 2,
    'activation': 'relu',
    'use_norm': True,
    'num_groups': 4,
    'variational': False,
    'lstm': False,
}

test_autoencoder_model = Autoencoder1d(**model_config)

input_tensor = torch.randn(7, 1, 44025)

x_tensor = test_autoencoder_model(input_tensor)
x_tensor['x'] = input_tensor

test_loss.backward(x_tensor)

{'generator_loss': 0.597416341304779,
 'feature_loss': 6.265276908874512,
 'l1_loss': 8.513725280761719,
 'l2_loss': 113.57705688476562,
 'aura_loss': 12.271647453308105,
 'global_loss': 141.22512286901474}

In [11]:
x_tensor['x'].shape

torch.Size([7, 1, 44025])

In [12]:
x_tensor['x_hat'].shape

torch.Size([7, 1, 44025])